In [1]:
import numpy as np
import tensorflow as tf
import scipy.io as sio

In [2]:
train_x=sio.loadmat('E:\\Biyun Yan\\wpi\machine learning in cyber scurity\\pro5\\data\\Training_data_40classes_Tor_Browser.mat')
train_data=train_x['X_train3']
train_y=sio.loadmat('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro5\\data\\Training_label_40classes_Tor_Browser.mat')
train_label=train_y['Ytrain'].T-1

In [3]:
test_x=sio.loadmat('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro5\\data\\Test_data_40classes_Tor_Browser.mat')
test_data=test_x['X_test']
test_y=sio.loadmat('E:\\Biyun Yan\\wpi\\machine learning in cyber scurity\\pro5\\data\\Test_label_40classes_Tor_Browser.mat')
test_label=test_y['Ytest'].T-1

In [4]:
x_placeholder=tf.placeholder(tf.float32,[None,100,60,1])
y_placeholder=tf.placeholder(tf.int32,[None,1])
conv0=tf.layers.conv2d(x_placeholder,200,10,activation=tf.nn.relu,padding='same')
pool0=tf.layers.max_pooling2d(conv0,[10,6],[2,2])
conv1=tf.layers.conv2d(pool0,100,4,activation=tf.nn.relu,padding='same')
pool1=tf.layers.max_pooling2d(conv1,[10,6],[2,2])
#conv2=tf.layers.conv2d(pool0,60,2,activation=tf.nn.relu,padding='same')
#pool2=tf.layers.max_pooling2d(conv2,[2,2],[2,2])

flatten=tf.layers.flatten(pool1)
print(flatten)
#fc=tf.layers.dense(flatten,20,activation=tf.nn.relu)
logits=tf.layers.dense(flatten,40)
predicted_labels=tf.arg_max(logits,1)
losses=tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y_placeholder,40),logits=logits)
mean_loss=tf.reduce_mean(losses)
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(losses)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Tensor("flatten/Reshape:0", shape=(?, 22800), dtype=float32)
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use `tf.math.argmax` instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [5]:
train_x=np.reshape(train_data,[-1,100,60,1])
train_y=train_label
test_x=np.reshape(test_data,[-1,100,60,1])
test_y=test_label

In [ ]:
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #val_feed_dict={x_placeholder:val_x,y_placeholder:val_y}
    train_feed_dict={x_placeholder:train_x,y_placeholder:train_y}
    max_acc=0
    for step in range(10):
        _,mean_loss_val,predicted_labels_train=sess.run([optimizer,mean_loss,predicted_labels],feed_dict=train_feed_dict)
       
        #predicted_labels_validation=sess.run([predicted_labels],feed_dict=val_feed_dict)
        acc=np.sum(predicted_labels_train==train_y)/train_x.shape[0]
        print(step,mean_loss_val,acc)
        if max_acc<acc:
            max_acc=acc
            print("saving model")
            saver.save(sess,'model_cnn/')

In [24]:
with tf.Session() as sess:
    saver.restore(sess,'model_cnn/')
    test_feed_dict={x_placeholder:test_x,y_placeholder:test_y}
    predicted_labels_val=sess.run(predicted_labels,feed_dict=test_feed_dict)
    acc=np.sum(predicted_labels_val==test_y)/test_x.shape[0]
    print(acc)

INFO:tensorflow:Restoring parameters from model_cnn/
10.0
